## Verwendete Biblotheken 

In [ ]:
import couchdb
import json 
import os 
import time 
from pathlib import Path
from dotenv import load_dotenv


Zweck: Den "Gesprächskanal" zwischen Python-Code und dem CouchDB-Server im Docker-Container herstellen und sicherstellen, dass die Zieldatenbank existiert.

In [ ]:
# --- VERBINDUNGS-PARAMETER ---
# CouchDB-Benutzer und Passwort aus Umgebungsvariablen laden
load_dotenv(dotenv_path='.env')
COUCHDB_USER = os.getenv("COUCHDB_USER")
COUCHDB_PASSWORD = os.getenv("COUCHDB_PASSWORD")
COUCHDB_HOST = "localhost:5984" 
COUCHDB_URL = f"http://{COUCHDB_USER}:{COUCHDB_PASSWORD}@{COUCHDB_HOST}"
DB_NAME = 'world_factbook' 

try:
    server = couchdb.Server(COUCHDB_URL)
    print(f"Erfolgreich mit CouchDB Server verbunden: {server}")

    if DB_NAME in server:
        db = server[DB_NAME]
        print(f"Datenbank '{DB_NAME}' existiert bereits.")
    else:
        db = server.create(DB_NAME)
        print(f"Datenbank '{DB_NAME}' wurde neu erstellt.")

except Exception as e:
    print(f"FEHLER bei der Verbindung oder Datenbankerstellung: {e}")
    raise e 

Zweck: Die Rohdaten aus der countries.json-Datei von der Festplatte in den Arbeitsspeicher (RAM) des Computers laden und so aufbereiten.

In [ ]:
base_dir = Path.cwd()
if base_dir.name == "notebooks":
    base_dir = base_dir.parent 
json_path = base_dir / "data" / "countries.json"

if not os.path.exists(json_path):
     print(f"FEHLER: JSON-Datei nicht gefunden unter: {json_path}")
     raise FileNotFoundError (f"JSON-Datei nicht gefunden unter: {json_path}")
else:
    print(f"JSON-Datei gefunden unter: {json_path}")

try:
    with open(json_path, 'r', encoding='utf-8') as f:
        loaded_data = json.load(f)

    if isinstance(loaded_data, dict):
        print("JSON-Datei erfolgreich als Dictionary geladen (Ländernamen als Keys).")
        countries_data = list(loaded_data.values()) 
        total_docs_to_load = len(countries_data)
        print(f"Die Länderdaten wurden in eine Liste umgewandelt. {total_docs_to_load} Länder-Objekte gefunden.")

except json.JSONDecodeError as e:
    print(f"FEHLER beim Parsen der JSON-Datei: {e}")
    raise e
except Exception as e:
    print(f"FEHLER beim Lesen oder Verarbeiten der JSON-Datei: {e}")
    raise e

Zweck: Die im RAM vorbereiteten Länderdaten (aus countries_data) nehmen und dauerhaft als einzelne Dokumente in der CouchDB-Datenbank speichern.


In [ ]:
docs_uploaded = 0
docs_failed = 0

if not 'countries_data' in locals() or not isinstance(countries_data, list):
    print("FEHLER: 'countries_data' nicht korrekt geladen.")
else:
    total_docs = len(countries_data)
    existing_docs = len(db) 
    if existing_docs >= total_docs:
        print(f"Datenbank '{DB_NAME}' enthält bereits {existing_docs} Dokumente. Import wird übersprungen.")
    else:
        print(f"\nStarte das Speichern von {total_docs} Dokumenten in CouchDB ('{DB_NAME}')...")
        start_time = time.time()

        for i, country_doc in enumerate(countries_data):
            try:
                country_doc['type'] = 'country'
                country_doc.pop('_id', None)

                db.save(country_doc)
                docs_uploaded += 1

            except Exception as e:
                print(f"\nFEHLER beim Speichern von Dokument {i} ('{country_doc.get('Country', 'N/A')}') : {e}")
                docs_failed += 1

            if (i + 1) % 100 == 0 or (i + 1) == total_docs:
                elapsed_time = time.time() - start_time
                print(f"  Verarbeitet: {i + 1}/{total_docs} | Gespeichert: {docs_uploaded} | Fehler: {docs_failed} | Zeit: {elapsed_time:.2f}s")

        end_time = time.time()
        print(f"Gesamtzeit: {end_time - start_time:.2f}s")
        print(f"Erfolgreich gespeicherte Dokumente: {docs_uploaded}")
        print(f"Fehlgeschlagene Dokumente: {docs_failed}")

    time.sleep(2)
    try:
        db = server[DB_NAME]
        final_doc_count = len(db) 
        print(f"\nAnzahl Dokumente in DB '{DB_NAME}': {final_doc_count}")
    except Exception as e:
        print(f"Fehler beim Abrufen der finalen Dokumentenanzahl: {e}")